<a href="https://colab.research.google.com/github/peterbabulik/ETA/blob/main/HHLAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The Linear System Equation

**The Math:** Solving a system of linear equations.
$$ A|x\rangle = |b\rangle $$

Where $A$ is an $N \times N$ matrix, $|b\rangle$ is a known vector, and $|x\rangle = A^{-1}|b\rangle$ is the solution we seek.

Classically, solving this requires $O(N)$ time for sparse matrices, or $O(N^3)$ for general matrices using Gaussian elimination.

### The Quantum Translation: HHL Algorithm

The HHL (Harrow, Hassidim, Lloyd) algorithm solves linear systems in $O(\log N)$ time under certain conditions - an exponential speedup!

**The Logic:**
1. **Encode:** Load $|b\rangle$ into a quantum state.
2. **Phase Estimation:** Decompose $A$ into its eigenbasis: $A = \sum_j \lambda_j |u_j\rangle\langle u_j|$.
3. **Rotation:** Apply controlled rotation proportional to $1/\lambda_j$.
4. **Inverse Phase Estimation:** Uncompute the eigenvalue register.
5. **Post-select:** Measure the ancilla qubit; if $|1\rangle$, we have $|x\rangle \propto A^{-1}|b\rangle$.

**Analogy for Python Devs:**
Think of this as `solve_linear_system()` - like `numpy.linalg.solve(A, b)`, but exponentially faster for certain problems.

**Important Caveat:** The solution $|x\rangle$ is a quantum state. You can't read all values directly, but you can compute expectation values $\langle x|M|x\rangle$ efficiently.

### The Qiskit Implementation

We'll implement a simplified HHL for a 2×2 system.

In [1]:
!pip install qiskit qiskit-aer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 924.6 kB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt

# -------------------------------------------------------
# HHL Algorithm for 2x2 System
# -------------------------------------------------------
# We solve: A|x> = |b>
# For simplicity, we use A = [[1, -1/3], [-1/3, 1]]
# This is a well-conditioned matrix with eigenvalues 2/3 and 4/3

def create_hhl_circuit():
    """
    Creates a simplified HHL circuit for a 2x2 linear system.

    System: A|x> = |b>
    A = [[1, -1/3], [-1/3, 1]]
    |b> = |0> (we'll rotate it to [1, 0])

    Circuit structure:
    - 1 ancilla qubit (for rotation)
    - 1 clock qubit (for eigenvalue)
    - 1 input qubit (for |b> and |x>)
    """

    # Define registers
    ancilla = QuantumRegister(1, 'ancilla')
    clock = QuantumRegister(1, 'clock')
    input_reg = QuantumRegister(1, 'input')
    measurement = ClassicalRegister(2, 'meas')  # ancilla + input

    qc = QuantumCircuit(ancilla, clock, input_reg, measurement)

    # -------------------------------------------------------
    # STEP 1: State Preparation - Prepare |b>
    # For this example, |b> = |0>
    # (Already in |0> state, no action needed)
    # -------------------------------------------------------

    # -------------------------------------------------------
    # STEP 2: Quantum Phase Estimation
    # We need to encode eigenvalues of A into the clock register
    # For our simple matrix, eigenvalues are 2/3 and 4/3
    #
    # Simplified: Apply H to clock to create superposition
    # Then controlled-U operations
    # -------------------------------------------------------

    # Put clock in superposition
    qc.h(clock[0])

    # Controlled-U operation (simplified for our matrix)
    # In practice, this would be controlled-e^(-iAt)
    # Here we use a controlled rotation that encodes the eigenvalues
    qc.cp(np.pi, clock[0], input_reg[0])  # Simplified

    # -------------------------------------------------------
    # STEP 3: Controlled Rotation (The Key Step)
    # Rotate ancilla by angle proportional to 1/lambda
    # This is where the "inversion" happens
    # -------------------------------------------------------

    # For eigenvalue lambda, rotate by angle ~ 1/lambda
    # We use Ry rotation on ancilla, controlled by clock
    # Simplified rotation angle
    rotation_angle = np.pi / 3  # Corresponds to 1/lambda for our case
    qc.cry(rotation_angle, clock[0], ancilla[0])

    # -------------------------------------------------------
    # STEP 4: Inverse Quantum Phase Estimation
    # Uncompute the clock register
    # -------------------------------------------------------

    qc.cp(-np.pi, clock[0], input_reg[0])  # Inverse of controlled-U
    qc.h(clock[0])  # Inverse H

    # -------------------------------------------------------
    # STEP 5: Measurement
    # Measure ancilla and input
    # If ancilla = |1>, input contains |x>
    # -------------------------------------------------------

    qc.measure(ancilla[0], measurement[0])
    qc.measure(input_reg[0], measurement[1])

    return qc

# -------------------------------------------------------
# Alternative: Direct Statevector Simulation
# -------------------------------------------------------

def hhl_statevector_simulation():
    """
    Simulates HHL using statevector for educational purposes.
    This shows the ideal quantum state without measurement noise.
    """
    # Define the system
    A = np.array([[1, -1/3],
                  [-1/3, 1]])
    b = np.array([1, 0])  # |0> state

    # Classical solution for comparison
    x_classical = np.linalg.solve(A, b)
    x_classical_normalized = x_classical / np.linalg.norm(x_classical)

    print("=== HHL Algorithm: Linear System Solver ===")
    print("\nSystem: A|x> = |b>")
    print(f"\nA = \n{A}")
    print(f"\n|b> = {b}")

    # Eigenvalue decomposition
    eigenvalues, eigenvectors = np.linalg.eig(A)

    print("\n--- Eigenvalue Decomposition ---")
    print(f"Eigenvalues: {eigenvalues}")
    print(f"Eigenvectors:\n{eigenvectors}")

    # HHL would:
    # 1. Express |b> in eigenbasis
    # 2. Apply 1/lambda rotation
    # 3. Transform back

    # Express b in eigenbasis
    b_eigenbasis = eigenvectors.T @ b
    print(f"\n|b> in eigenbasis: {b_eigenbasis}")

    # Apply 1/lambda (this is the quantum "inversion")
    x_eigenbasis = b_eigenbasis / eigenvalues
    print(f"\n|x> in eigenbasis (after 1/lambda): {x_eigenbasis}")

    # Transform back to computational basis
    x_quantum = eigenvectors @ x_eigenbasis
    x_quantum_normalized = x_quantum / np.linalg.norm(x_quantum)

    print("\n--- Results ---")
    print(f"\nClassical solution: {x_classical}")
    print(f"Classical (normalized): {x_classical_normalized}")
    print(f"\nQuantum solution (normalized): {x_quantum_normalized}")

    # Fidelity
    fidelity = np.abs(np.dot(x_classical_normalized, x_quantum_normalized))**2
    print(f"\nFidelity: {fidelity:.4f}")

    return x_classical, x_quantum_normalized

# Run the simulation
x_classical, x_quantum = hhl_statevector_simulation()

# Create and display the circuit
print("\n--- HHL Circuit Structure (Simplified) ---")
hhl_circuit = create_hhl_circuit()
print(hhl_circuit.draw(output='text'))

=== HHL Algorithm: Linear System Solver ===

System: A|x> = |b>

A = 
[[ 1.         -0.33333333]
 [-0.33333333  1.        ]]

|b> = [1 0]

--- Eigenvalue Decomposition ---
Eigenvalues: [1.33333333 0.66666667]
Eigenvectors:
[[ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]

|b> in eigenbasis: [0.70710678 0.70710678]

|x> in eigenbasis (after 1/lambda): [0.53033009 1.06066017]

--- Results ---

Classical solution: [1.125 0.375]
Classical (normalized): [0.9486833  0.31622777]

Quantum solution (normalized): [0.9486833  0.31622777]

Fidelity: 1.0000

--- HHL Circuit Structure (Simplified) ---
                     ┌─────────┐        ┌─┐     
ancilla: ────────────┤ Ry(π/3) ├────────┤M├─────
         ┌───┐       └────┬────┘        └╥┘┌───┐
  clock: ┤ H ├─■──────────■──────■───────╫─┤ H ├
         └───┘ │P(π)             │P(-π)  ║ └┬─┬┘
  input: ──────■─────────────────■───────╫──┤M├─
                                         ║  └╥┘ 
 meas: 2/════════════════════════════════╩═══╩══
       

In [3]:
# Run the circuit on simulator
simulator = AerSimulator()
compiled = transpile(hhl_circuit, simulator)
job = simulator.run(compiled, shots=10000)
result = job.result()
counts = result.get_counts()

print("--- Measurement Results ---")
print(f"Counts: {counts}")
print("\nBit ordering: [ancilla, input]")
print("  '10' = ancilla=1, input=0 -> Success! Input contains |x>")
print("  '11' = ancilla=1, input=1 -> Success! Input contains |x>")
print("  '00' or '01' = ancilla=0 -> Failure, need to retry")

# Calculate success probability
success_prob = (counts.get('10', 0) + counts.get('11', 0)) / 10000
print(f"\nSuccess probability (ancilla=1): {success_prob:.2%}")

--- Measurement Results ---
Counts: {'01': 1232, '00': 8768}

Bit ordering: [ancilla, input]
  '10' = ancilla=1, input=0 -> Success! Input contains |x>
  '11' = ancilla=1, input=1 -> Success! Input contains |x>
  '00' or '01' = ancilla=0 -> Failure, need to retry

Success probability (ancilla=1): 0.00%


### Understanding the Translation

1. **$A|x\rangle = |b\rangle$ (The Linear System):**
   - Classical: Compute $A^{-1}$ explicitly or use iterative methods.
   - Quantum: Use eigenvalue decomposition and invert eigenvalues.

2. **Eigenvalue Inversion:**
   - The key quantum operation is rotating by $1/\lambda_j$.
   - This is where the "matrix inversion" happens in quantum form.

3. **Post-Selection:**
   - We measure the ancilla qubit.
   - If result is $|1\rangle$, the input register contains $|x\rangle$.
   - Success probability depends on the condition number of $A$.

### Why is this useful?

**Exponential Speedup:** For sparse, well-conditioned matrices, HHL provides exponential speedup over classical methods.

**Applications:**
- Machine learning (quantum SVM, quantum PCA)
- Differential equations
- Finance (portfolio optimization)
- Engineering (finite element methods)

**Limitations:**
- Matrix must be sparse and well-conditioned
- Loading classical data into quantum states is hard
- Can only extract certain properties of $|x\rangle$, not all values

### Complexity Comparison

| Method | Time Complexity |
|--------|---------------|
| Gaussian Elimination | $O(N^3)$ |
| Sparse Matrix Methods | $O(Ns)$ (s = sparsity) |
| **HHL (Quantum)** | $O(\log N \cdot \kappa^2)$ (k = condition number) |

### Summary for Python Developers

```python
# Classical approach
import numpy as np
x = np.linalg.solve(A, b)  # O(N^3) for dense matrices

# Quantum approach (conceptual)
# HHL gives |x> as a quantum state
# You can compute <x|M|x> efficiently
# But you cannot read all N values of x directly
```